# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770626965992                   -0.53    9.0         
  2   -2.772138746419       -2.82       -1.29    1.0    237ms
  3   -2.772170011273       -4.50       -2.69    1.0    177ms
  4   -2.772170720769       -6.15       -3.86    2.0    208ms
  5   -2.772170722907       -8.67       -4.46    1.0    218ms
  6   -2.772170723007      -10.00       -5.34    1.0    182ms
  7   -2.772170723014      -11.13       -5.64    2.0    225ms
  8   -2.772170723015      -12.25       -6.93    1.0    190ms
  9   -2.772170723015      -13.66       -7.01    2.0    233ms
 10   -2.772170723015   +  -13.89       -7.31    1.0    203ms
 11   -2.772170723015      -14.07       -8.44    1.0    205ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770748343173                   -0.52    9.0         
  2   -2.7

1.773558026471175

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770834309580                   -0.52    9.0         
  2   -2.772062557049       -2.91       -1.32    1.0    173ms
  3   -2.772083048247       -4.69       -2.45    1.0    232ms
  4   -2.772083343147       -6.53       -3.15    1.0    179ms
  5   -2.772083417329       -7.13       -4.14    2.0    252ms
  6   -2.772083417626       -9.53       -4.42    1.0    182ms
  7   -2.772083417808       -9.74       -5.73    1.0    207ms
  8   -2.772083417810      -11.59       -6.02    2.0    211ms
  9   -2.772083417811      -12.97       -6.47    1.0    210ms
 10   -2.772083417811      -13.75       -7.96    2.0    224ms
 11   -2.772083417811   +  -14.57       -8.43    2.0    233ms

Polarizability via ForwardDiff:       1.7725349763338558
Polarizability via finite difference: 1.773558026471175
